In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-04-29"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
9720,2024-04-29,Romênia Divisão A,13:00,Dinamo Bucharest,Rapid Bucuresti,1.69,2.06,151.5,1.85,1.85,-2.5,1.94,2.50,4zQPYf3L,0.591716,0.485437,0.540541,0.540541,0.077153,365.110,242.167598,0.663273,0.6,1.341641,2.236068,185.50,5.592,3.677407,0.657619,-86.0,115.968,17.819302,0.153657,1.2,1.643168,1.369306,115.20,1.618,0.401398,0.248083,-2.0,70,80,2.65,1.44,243.614,193.962,0.139536,0.000000,0.178369,0.02,0.004,172.500000,0.463986,0.6,0.136014,-1.28,-0.256,-4.140625,0.506844,0.4,-0.106844
9721,2024-04-29,Letônia Lbl,13:30,Rigas Zelli,Ventspils,1.75,2.01,169.5,1.87,1.79,-2.5,2.01,2.40,G2aRLnyG,0.571429,0.497512,0.534759,0.558659,0.068941,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,81,0,1.61,0.00,0.000,0.000,0.097791,0.030912,0.125067,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
9722,2024-04-29,Libano Divisão 1,15:45,Beirut Club,Sagesse Al Hekmeh Beirut,1.70,2.05,172.5,1.85,1.85,-2.5,1.95,2.49,z53VF9Lk,0.588235,0.487805,0.540541,0.540541,0.076040,137.850,75.570977,0.548212,1.8,1.643168,0.912871,104.86,1.468,0.819005,0.557906,53.0,191.198,77.002233,0.402736,1.8,1.643168,0.912871,107.12,2.058,0.973406,0.472987,36.0,98,103,1.07,1.04,138.052,202.638,0.131993,0.000000,0.171999,1.17,0.234,2.991453,0.000000,0.0,0.000000,-2.55,-0.510,-2.058824,0.618324,0.5,-0.118324
9723,2024-04-29,Lituânia Lkl,12:50,Zalgiris Kaunas,Mazeikiai,1.02,11.60,159.5,1.87,1.79,0.0,0.00,0.00,dfxkDvlp,0.980392,0.086207,0.534759,0.558659,0.066599,111.592,13.120736,0.117578,2.4,1.341641,0.559017,132.16,1.176,0.111714,0.094995,61.0,207.392,133.186067,0.642195,1.2,1.643168,1.369306,84.94,2.650,1.583398,0.597509,-37.0,112,62,1.18,1.37,113.104,266.576,1.185609,0.030912,NaN,0.78,0.156,0.128205,0.000000,0.0,0.000000,-0.93,-0.186,-56.989247,0.000000,0.0,0.000000
9724,2024-04-29,Paraguai Lnb,20:30,Ciudad Nueva,Colonias Gold,13.00,1.01,155.5,1.85,1.85,0.0,0.00,0.00,UHa8MsYb,0.076923,0.990099,0.540541,0.540541,0.067022,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0,0,0.00,0.00,0.000,0.000,1.210308,0.000000,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
9725,2024-04-29,Paraguai Lnb,20:30,Felix Perez Cardozo,Olimpia,4.47,1.18,154.5,1.85,1.85,0.0,0.00,0.00,GYcGKL2B,0.223714,0.847458,0.540541,0.540541,0.071171,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0,77,0.00,2.90,0.000,0.000,0.823498,0.000000,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
9726,2024-04-29,Romênia Divisão A,13:00,CSM Constanta,CSU Sibiu,1.21,4.12,167.5,1.84,1.86,0.0,0.00,0.00,pI5LC3Fa,0.826446,0.242718,0.543478,0.537634,0.069165,299.628,158.385408,0.528607,0.6,1.341641,2.236068,195.65,3.756,2.057020,0.547662,-46.0,235.468,105.584703,0.448404,1.8,1.643168,0.912871,153.45,2.648,1.035529,0.391061,-2.0,91,93,2.15,1.65,198.336,270.220,0.772113,0.007644,NaN,-2.20,-0.440,-0.477273,0.579569,0.5,-0.079569,-1.47,-0.294,-10.612245,0.375014,0.3,-0.075014
9727,2024-04-29,Romênia Divisão A,13:00,B

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
9732,21:30,Eua Nba,New Orleans Pelicans,Oklahoma City Thunder,2.61,Back Home
9740,09:30,Irã Superliga,Zob Ahan,Tabiat,1.74,Back Home
